<b> Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#Matplot to graph
from matplotlib import pyplot as plt
%matplotlib inline

print('Libraries imported.')

Libraries imported.


In [2]:
df = pd.read_csv("/Users/kyleoddson/coursera/coursera/toronto2.csv")

In [3]:
df.head()

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [4]:
df.groupby("Borough").count()

PostalCode  Neighborhood  Latitude  Longitude
Borough                                                        
Central Toronto            9             9         9          9
Downtown Toronto          19            19        19         19
East Toronto               5             5         5          5
East York                  5             5         5          5
Etobicoke                 12            12        12         12
Mississauga                1             1         1          1
North York                24            24        24         24
Scarborough               17            17        17         17
West Toronto               6             6         6          6
York                       5             5         5          5

<b> Only going to work with the "... Toronto" boroughs

In [5]:
Tor = df[df["Borough"].isin(["Central Toronto", "Downtown Toronto", "East Toronto"])]

In [6]:
Tor.head()

PostalCode           Borough                                 Neighborhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  
15  43.651494 -79.375418  
19  43.676357 -79.293031

In [7]:
Tor.shape

(33, 5)

In [8]:
Tor.reset_index(inplace=True, drop=True)

In [9]:
Tor.head()

PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Tor['Borough'].unique()),
        Tor.shape[0]
    )
)

The dataframe has 3 boroughs and 33 neighborhoods.


<b> Use geopy library to get the latitude and longitude values of Toronto.
Define an instance of the geocoder. We will name our agent my_toronto.

In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="my_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<b> Create a map of Toronto with neighborhoods superimposed on top

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Tor['Latitude'], Tor['Longitude'], Tor['Borough'], Tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<b> Private Foursquare credentials will be read from a file

In [13]:
foursq = pd.read_csv('/Users/kyleoddson/coursera/coursera/foursqu.csv')
CLIENT_ID = foursq.columns[0]
CLIENT_SECRET = foursq.columns[1]
VERSION = foursq.columns[2]

<b> Create a function to get venues for all the neighborhoods in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b> Apply this function to all the neighborhoods and create a new dataframe called toronto_venues

In [15]:
LIMIT = 100
toronto_venues = getNearbyVenues(names=Tor['Neighborhood'],
                                   latitudes=Tor['Latitude'],
                                   longitudes=Tor['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
University of Toronto, Harbord
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
Business reply mail Processing CentrE


<b> Check size of resulting dataframe

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(1465, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4      Morning Glory Cafe       43.653947       -79.361149   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4       Breakfast Spot

<b> Check how many venues returned for each neighborhood

In [17]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            57   
Business reply mail Processing CentrE                                  17   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     65   
Christie                                                               18   
Church and Wellesley                                                   72   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        7   
First Canadian Place, Underground city                                100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
India Bazaar, The Beaches West                                         21   
Kensington Market, Chinatown, Grange Park                              56   
Lawrence Park                                                           3   
Moore Park, Summerhill East                                             4   
North Toronto West                                                     20   
Queen's Park, Ontario Provincial Government                            38   
Regent Park, Harbourfront                                              46   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                2   
St. James Town                                                         84   
St. James Town, Cabbagetown                                            46   
Stn A PO Boxes                                                         97   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    22   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         35   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             57   
Business reply mail Processing CentrE                                   17   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      65   
Christie                                                                18   
Church and Wellesley                                                    72   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              34   
Davisville North                                                         7   
First Canadian Place, Underground city                                 100   
Forest Hill North & West                                                 4   
Garden District, Ryerson                                               100   
Harbourfront East, Union Station, Toronto Islands                      100   
India Bazaar, The Beaches West                                          21   
Kensington Market, Chinatown, Grange Park                               56   
Lawrence Park                 

<b>Analyze each neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


In [19]:
# find which column contains Neighborhoods
for i, col in enumerate(list(toronto_onehot)):
    if col == "Neighborhood":
        print(i)

154


In [20]:
# Move Neighborhood column to first column
fixed_columns = [toronto_onehot.columns[154]] + list(toronto_onehot.columns[:154]) + list(toronto_onehot.columns[155:])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

Neighborhood  Airport  Airport Food Court  Airport Gate  \
0  Regent Park, Harbourfront        0                   0             0   
1  Regent Park, Harbourfront        0                   0             0   
2  Regent Park, Harbourfront        0                   0             0   
3  Regent Park, Harbourfront        0                   0             0   
4  Regent Park, Harbourfront        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  \
0                 0                   0              0          0           0   
1                 0                   0              0          0           0   
2                 0                   0              0          0           0   
3                 0                   0              0          0           0   
4                 0                   0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  \
0           0       1     0    0                 0                   0      0   
1           0       0     0    0                 0                   0      0   
2           0       0     0    0                 0                   0      0   
3           0       0     0    0                 0                   0      0   
4           0       0     0    0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bistro  \
0                0         0           0                   0       0   
1                0         0           0                   0       0   
2                0         0           0                   0       0   
3                0         0           0                   0       0   
4                0         0           0                   0       0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0                     0               1   

   Brewery  Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0        0                0         0             0              0         0   
1        0                0         0             0              0         0   
2        0                0         0             0              0         0   
3        0                0         0             0              0         0   
4        0                0         0             0              0         0   

   Business Service  Butcher  Café  Candy Store  Caribbean Restaurant  \
0                 0        0     0            0                     0   
1                 0        0     0            0                     0   
2                 0        0     0            0                     0

In [21]:
toronto_onehot.shape

(1465, 222)

<b> Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Airport  \
0                                         Berczy Park   0.0000   
1               Business reply mail Processing CentrE   0.0000   
2   CN Tower, King and Spadina, Railway Lands, Har...   0.0625   
3                                  Central Bay Street   0.0000   
4                                            Christie   0.0000   
5                                Church and Wellesley   0.0000   
6                      Commerce Court, Victoria Hotel   0.0000   
7                                          Davisville   0.0000   
8                                    Davisville North   0.0000   
9              First Canadian Place, Underground city   0.0000   
10                           Forest Hill North & West   0.0000   
11                           Garden District, Ryerson   0.0000   
12  Harbourfront East, Union Station, Toronto Islands   0.0000   
13                     India Bazaar, The Beaches West   0.0000   
14          Kensington Market, Chinatown, Grange Park   0.0000   
15                                      Lawrence Park   0.0000   
16                        Moore Park, Summerhill East   0.0000   
17                                 North Toronto West   0.0000   
18        Queen's Park, Ontario Provincial Government   0.0000   
19                          Regent Park, Harbourfront   0.0000   
20                           Richmond, Adelaide, King   0.0000   
21                                           Rosedale   0.0000   
22                                           Roselawn   0.0000   
23                                     St. James Town   0.0000   
24                        St. James Town, Cabbagetown   0.0000   
25                                     Stn A PO Boxes   0.0000   
26                                    Studio District   0.0000   
27  Summerhill West, Rathnelly, South Hill, Forest...   0.0000   
28                The Annex, North Midtown, Yorkville   0.0000   
29                                        The Beaches   0.0000   
30                       The Danforth West, Riverdale   0.0000   
31           Toronto Dominion Centre, Design Exchange   0.0000   
32                     University of Toronto, Harbord   0.0000   

    Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0               0.0000        0.0000           0.000            0.000   
1               0.0000        0.0000           0.000            0.000   
2               0.0625        0.0625           0.125            0.125   
3               0.0000        0.0000           0.000            0.000   
4               0.0000        0.0000           0.000            0.000   
5               0.0000        0.0000           0.000            0.000   
6               0.0000        0.0000           0.000            0.000   
7               0.0000        0.0000           0.000            0.000   
8               0.0000        0.0000           0.000            0.000   
9               0.0000        0.0000           0.000            0.000   
10              0.0000        0.0000           0.000            0.000   
11              0.0000        0.0000           0.000            0.000   
12              0.0000        0.0000           0.000            0.000   
13              0.0000        0.0000           0.000            0.000   
14              0.0000        0.0000           0.000            0.000   
15              0.0000        0.0000           0.000            0.000   
16              0.0000        0.0000           0.000            0.000   
17              0.0000        0.0000           0.000            0.000   
18              0.0000        0.0000           0.000            0.000   
19              0.0000        0.0000           0.000            0.000   
20              0.0000        0.0000           0.000            0.000   
21              0.0000        0.0000           0.000            0.000   
22              0.0000        0.0000           0.000            0.000   
23              0.0000        0.0000        

<b> Print each neighborhood along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print(hood+": ")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Berczy Park: 
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3                Café  0.04
4         Cheese Shop  0.04


Business reply mail Processing CentrE: 
              venue  freq
0        Smoke Shop  0.06
1     Garden Center  0.06
2       Pizza Place  0.06
3        Comic Shop  0.06
4  Recording Studio  0.06


CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport: 
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3             Airport  0.06
4  Airport Food Court  0.06


Central Bay Street: 
                 venue  freq
0          Coffee Shop  0.20
1                 Café  0.06
2   Italian Restaurant  0.06
3       Sandwich Place  0.05
4  Japanese Restaurant  0.05


Christie: 
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3          Diner  0.06
4    Candy Store

<b> Build this into a dataframe, first sorting venues in descending order

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b> Build the dataframe and display top 10 venues for each neighborhood

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1              Business reply mail Processing CentrE            Skate Park   
2  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
3                                 Central Bay Street           Coffee Shop   
4                                           Christie         Grocery Store   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar              Beer Bar        Farmers Market   
1      Recording Studio               Butcher        Farmers Market   
2       Airport Service      Airport Terminal               Airport   
3    Italian Restaurant                  Café        Sandwich Place   
4                  Café                  Park                 Diner   

  5th Most Common Venue 6th Most Common Venue      7th Most Common Venue  \
0                Bakery    Italian Restaurant                Cheese Shop   
1    Light Rail Station         Burrito Place              Auto Workshop   
2       Harbor / Marina           Coffee Shop                      Plane   
3   Japanese Restaurant        Ice Cream Shop  Middle Eastern Restaurant   
4           Coffee Shop    Italian Restaurant                 Baby Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                  Café            Restaurant     Seafood Restaurant  
1            Restaurant                   Spa            Pizza Place  
2   Rental Car Location      Sculpture Garden               Boutique  
3   Fried Chicken Joint           Salad Place           Burger Joint  
4            Restaurant    Athletics & Sports              Nightclub

<b> Determine the optimal number of clusters (elbow method):

In [26]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

elbow = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300,
                   n_init = 10, random_state = 0)
    kmeans.fit(toronto_grouped_clustering)
    elbow.append(kmeans.inertia_)
plt.plot(range(1,11), elbow)
plt.title('Finding Optimal Clusters')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

<b> Since there is little change in Within Cluster Sum of Squares (WCSS) for more than 5 clusters, we will use K = 5

In [27]:
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

<b> Create a new dataframe for visualization

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Tor

# merge toronto_grouped with Tor to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               3           Coffee Shop   
1  43.662301 -79.389494               3           Coffee Shop   
2  43.657162 -79.378937               3        Clothing Store   
3  43.651494 -79.375418               3           Coffee Shop   
4  43.676357 -79.293031               0                 Trail   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Park                Bakery                   Pub   
1      Sushi Restaurant                 Diner           Yoga Studio   
2           Coffee Shop                  Café            Restaurant   
3                  Café   American Restaurant             Gastropub   
4     Health Food Store                   Pub           Yoga Studio   

  5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
0        Breakfast Spot                   Restaurant                  Café   
1           Music Venue                     Beer Bar        Sandwich Place   
2        Cosmetics Shop          Japanese Restaurant       Bubble Tea Shop   
3          Cocktail Bar           Italian Restaurant                 Hotel   
4          Dance Studio  Eastern European Restaurant   Dumpling Restaurant   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                    Theater    Mexican Restaurant             Shoe Store  
1               Burger Joint         Burrito Place                   Café  
2  Middle Eastern Restaurant      Ramen Restaurant              Bookstore  
3         Seafood Restaurant            Restaurant               Creperie  
4                 Donut Shop      Doner Restaurant                Dog Run

<b> Visualize the clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b> Examine the clusters

<b> Cluster 1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


Neighborhood  Cluster Labels 1st Most Common Venue  \
4                The Beaches               0                 Trail   
18  Forest Hill North & West               0              Bus Line   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4      Health Food Store                   Pub           Yoga Studio   
18         Jewelry Store                 Trail      Sushi Restaurant   

   5th Most Common Venue        6th Most Common Venue  \
4           Dance Studio  Eastern European Restaurant   
18           Yoga Studio            Electronics Store   

          7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
4           Dumpling Restaurant            Donut Shop      Doner Restaurant   
18  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   10th Most Common Venue  
4                 Dog Run  
18       Doner Restaurant

<b> Cluster 2

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


Neighborhood  Cluster Labels 1st Most Common Venue  \
23  Moore Park, Summerhill East               1                  Park   
27                     Rosedale               1                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23            Playground                 Trail           Summer Camp   
27            Playground                 Trail          Dance Studio   

          5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23                  Yoga Studio          Dance Studio   Dumpling Restaurant   
27  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
23            Donut Shop      Doner Restaurant                Dog Run  
27      Doner Restaurant               Dog Run    Distribution Center

<b> Cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]



Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
16     Roselawn               2                Garden        Ice Cream Shop   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
16           Yoga Studio      Department Store     Electronics Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
16  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
16      Doner Restaurant                Dog Run

<b> Cluster 4

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]



Neighborhood  Cluster Labels  \
0                           Regent Park, Harbourfront               3   
1         Queen's Park, Ontario Provincial Government               3   
2                            Garden District, Ryerson               3   
3                                      St. James Town               3   
5                                         Berczy Park               3   
6                                  Central Bay Street               3   
7                                            Christie               3   
8                            Richmond, Adelaide, King               3   
9   Harbourfront East, Union Station, Toronto Islands               3   
10                       The Danforth West, Riverdale               3   
11           Toronto Dominion Centre, Design Exchange               3   
12                     India Bazaar, The Beaches West               3   
13                     Commerce Court, Victoria Hotel               3   
14                                    Studio District               3   
17                                   Davisville North               3   
19                                 North Toronto West               3   
20                The Annex, North Midtown, Yorkville               3   
21                                         Davisville               3   
22                     University of Toronto, Harbord               3   
24          Kensington Market, Chinatown, Grange Park               3   
25  Summerhill West, Rathnelly, South Hill, Forest...               3   
26  CN Tower, King and Spadina, Railway Lands, Har...               3   
28                                     Stn A PO Boxes               3   
29                        St. James Town, Cabbagetown               3   
30             First Canadian Place, Underground city               3   
31                               Church and Wellesley               3   
32              Business reply mail Processing CentrE               3   

   1st Most Common Venue 2nd Most Common Venue  3rd Most Common Venue  \
0            Coffee Shop                  Park                 Bakery   
1            Coffee Shop      Sushi Restaurant                  Diner   
2         Clothing Store           Coffee Shop                   Café   
3            Coffee Shop                  Café    American Restaurant   
5            Coffee Shop          Cocktail Bar               Beer Bar   
6            Coffee Shop    Italian Restaurant                   Café   
7          Grocery Store                  Café                   Park   
8            Coffee Shop                  Café             Restaurant   
9            Coffee Shop              Aquarium                   Café   
10      Greek Restaurant           Coffee Shop     Italian Restaurant   
11           Coffee Shop                 Hotel                   Café   
12                  Park        Sandwich Place   Fast Food Restaurant   
13           Coffee Shop            Restaurant                   Café   
14                  Café           Coffee Shop              Gastropub   
17                  Park                   Gym         Breakfast Spot   
19        Clothing Store           Coffee Shop    Sporting Goods Shop   
20        Sandwich Place                  Café            Coffee Shop   
21        Sandwich Place           Pizza Place           Dessert Shop   
22                  Café            Restaurant              Bookstore   
24                  Café           Coffee Shop  Vietnamese Restaurant   
25                   Pub           Coffee Shop     Light Rail Station   
26        Airport Lounge       Airport Service       Airport Terminal   
28           Coffee Shop    Italian Restaurant     Seafood Restaurant   
29           Coffee Shop                  Park             Restaurant   
30           Coffee Shop                  Café                  Hotel   
31      Sushi Restaurant           Coffee Shop    Japanese Restaurant   
32            Skate Park 

<b> Cluster 5

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]



Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
15  Lawrence Park               4                  Park           Swim School   

   3rd Most Common Venue 4th Most Common Venue        5th Most Common Venue  \
15              Bus Line  Ethiopian Restaurant  Eastern European Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
15   Dumpling Restaurant            Donut Shop      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
15               Dog Run    Distribution Center